In [3]:
import pandas as pd
import json
import ast

# ✅ CSV 불러오기
df_parsed = pd.read_csv("/Users/gim-yujin/Desktop/pjt_personal_agent/영양소 데이터/parsed_supplements_final.csv")

# ✅ final_parsed 컬럼이 문자열(list 형태의 str)이라면, 실제 리스트로 변환 필요
def convert_str_to_list(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return []
    return val if isinstance(val, list) else []

df_parsed["final_parsed"] = df_parsed["final_parsed"].apply(convert_str_to_list)

# ✅ RAG용 JSON 변환
def df_to_rag_json(df):
    rag_data = []

    for _, row in df.iterrows():
        entry = {
            "title": row["Title"],
            "category": [row.get("Category 1", ""), row.get("Category 2", ""), row.get("Category 3", "")],
            "brand": row.get("Brand", ""),
            "supplement_facts": row.get("final_parsed", []),
            "description": row.get("Description", ""),
            "tags": list({
                *(str(row.get("Category 1", "")).split()),
                *(str(row.get("Category 2", "")).split()),
                *(str(row.get("Category 3", "")).split()),
                *(i["name"] for i in row.get("final_parsed", []) if isinstance(i, dict) and "name" in i)
            })
        }
        rag_data.append(entry)
    
    return rag_data

# ✅ JSON 저장
rag_json = df_to_rag_json(df_parsed)
with open("supplement_rag_data_fixed.json", "w", encoding="utf-8") as f:
    json.dump(rag_json, f, ensure_ascii=False, indent=2)

print("📦 RAG용 JSON 파일 'supplement_rag_data_fixed.json' 저장 완료!")

📦 RAG용 JSON 파일 'supplement_rag_data_fixed.json' 저장 완료!
